In [454]:
import pymysql.cursors
import pandas as pd
import numpy as np
import connect_to_db as cn
import json
from itertools import combinations
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import csv
import parmap

In [455]:
# select query를 날려서 가져온 result를 DataFrame으로 반환하는 function.
def select_query_result_to_df(sql):
    connection = cn.make_connection('connection.csv')
    cursor = connection.cursor(pymysql.cursors.DictCursor)
    cursor.execute(sql)
    result = cursor.fetchall()
    connection.close()
    
    result_df = pd.DataFrame(result)
    
    return result_df

In [456]:
def check_body(document):
    p = re.compile('[a-zA-Z]')
    for text in document[:]:
        # 특수문자, 이모지 등으로만 이루어진 text를 거르기 위한 code.
        if p.search(text) is None:
            document.remove(text)
        # 특수문자와 알파벳 하나(ex. :p)로만 이루어진 text를 거르기 위한 code.
        elif len(re.sub('[^a-zA-Z]',' ',text).strip()) == 1:
            document.remove(text)
    
    return document

In [467]:
def tf_idf_similarity(sql, column_name):
    result_df = select_query_result_to_df(sql)
    document = list(np.array(result_df[column_name].tolist()))
    document = check_body(document)

    # comment/body list가 빈 list인지를 체크해서 빈 list이면 -1 return.
    if not document:
        return -1
    
    # check_body() 함수를 거친 후, 이모지로만 이루어진 comment/post title이 지워져서
    # 하나의 text만 남았을 경우. 구분을 위해 -2 return.
    if len(document) == 1:
        return -2

    tfidf_vectorizer = TfidfVectorizer(min_df=1)
    tfidf_matrix = tfidf_vectorizer.fit_transform(document)

    document_distances = (tfidf_matrix * tfidf_matrix.T)
    
    result_array = document_distances.toarray()
    
    similarity = 0
    count = 0
    
    for i in range(len(result_array)):
        for j in range(len(result_array[i])):
            if i < j:
                similarity += result_array[i][j]
                count += 1
    
    return (similarity / count)

In [472]:
def write_similarity_csv(filename, fields, result_list):
    with open(filename, 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(result_list)

In [469]:
def similarity_main(index):
    # similarity column들은 모두 null 값이라 가정하고 실행.
    sql = f'select author from mentor where comment_cnt > 1 and is_valid = 1;'
    result_df = select_query_result_to_df(sql)
    authors = np.array(result_df['author'].astype(str).values.tolist())

    result_for_csv = []
    
    if index % 100000 == 0:
        start_index = index - 100000
    else:
        start_index = index - (index % 100000)
        
    for i in range(start_index, index):
        sql1 = f"select body from comments where author = '{authors[i]}' and link_key = parent_key;"
        comments_similarity = tf_idf_similarity(sql1, 'body')
        
        sql2 = f"select distinct p.post_key, p.title from posts p, comments c where p.post_key = c.link_key and c.author = '{authors[i]}' and c.link_key = c.parent_key;";
        posts_similarity = tf_idf_similarity(sql2, 'title')        
        
        result_for_csv.append([authors[i], comments_similarity, posts_similarity])

    fields = ['author', 'comments_similarity', 'posts_similarity']
    write_similarity_csv(f'similarity_{index}.csv', fields, result_for_csv)

In [475]:
# index_list = [100000, 200000, 300000, 349103]
index_list = [100000, 200000, 227032]

if __name__ == '__main__':
    # multi processing.
    parmap.map(similarity_main, index_list, pm_pbar=True, pm_processes=3)

100%|██████████| 3/3 [11:57<00:00, 239.11s/it]
